. 

<font size=7>Opto Closed Loop Experiment Template

In this notebook we create the protocol for the opto close loop experiment. Make a copy of this template and the change the parameters to set up your specific experiment.

<font color="red"> **Before you run code** - Set key experiment variables below

In [6]:
maxwell_config = "/home/mxwbio/work/seizure_closed_loop/files/maxwell/configs/????????????.cfg"
#experiment_name = "hereitis" # Debug Code
arduino_path="/dev/ttyACM10"

In [ ]:
experiment_name = "???????????????"
#maxwell_config = "/home/mxwbio/work/seizure_closed_loop/files/maxwell/configs/2023_04_04_hc328_hckcr1-2.cfg" # Debug Code

# <font color="gray"> Set Up Notebook

Import required packages

In [8]:
import MaxwellGI as mgi
import time
import numpy as np
import matplotlib.pyplot as plt

## <font color="blue"> countSpikes

<font color="blue">countSpikes</font> is a function that counts the number of spikes that occure. Spike are counted over some set window of time, `count_window_seconds`. Spikes are continually counted throughout the duration of the experiment, `experiment_length_seconds`. All the spike counts that were recorded are then returned to the user.

In [9]:
def countSpikes( experiment_length_seconds, count_window_seconds=10 ):
    print("starting devices")
    maxwell = mgi.MaxwellCtl( config= maxwell_config )          # Turn Maxwell On
    stream  = mgi.Streamer( maxwell, filtered=False )           # Start Livestream
    
    print("Starting experiment")
    experiment_end = time.time() + experiment_length_seconds                                  
    first_time = True
    spike_counts = []
    while time.time() < experiment_end:                                  # while the experiment hasn't reached it's final run length
        count_time = time.time() + count_window_seconds                  # Count spikes over the count window length
        spike_count = 0                                                  # continually add new spikes to spike_count
        
        while time.time() < count_time:                                  # for our specified count winow length
            frameID, voltages, spikes = stream.getData()                 # get spikes data
            for spike in spikes:                                         # add up all of the spikes to spike_count
                spike_count += 1
        print("Spike Count: ", spike_count )    
        if not first_time:                                              # We don't record the first spike count, because that value is often wonky
            spike_counts.append(spike_count)                            # but we record the rest of the spike counts
        first_time = False

    
    print("Experiment over. Shutting down devices")
    del stream                # Stop Stream
    print("done")
    return spike_counts

## <font color="blue">optoRecording

In [10]:
def optoRecording( light_on_seconds, light_off_seconds, num_pulses, light_intensity,
                       calibration_file, data_file, log_file ):
    
    maxwell = mgi.MaxwellCtl( config=maxwell_config)                                       # Turn Maxwell On
    opto_ctl = mgi.OptoCtl( log_file=log_file , 
                       calibration_file=calibration_file, arduino_path=arduino_path)       # Set up Opto
    maxwell.recordingStart( data_file=data_file)                                           # Start recording data             

    print("Starting experiments")
    time.sleep(20)
    print("Light On")
    opto_ctl.pulses(  on_duration=light_on_seconds*1000, off_duration=light_off_seconds*1000, num_pulses=num_pulses, intensity=light_intensity)
    print("Light Off")
    time.sleep(20)

    # Shut Down Experiment
    print("shutting down devices")
    opto_ctl.close()          # Stop Opto
    maxwell.recordingStop()  # stop recording
    print("done")


## <font color="blue"> optoIntensitySweep

In [11]:
def optoIntensitySweep( light_on_seconds, calibration_file, data_file, log_file ):

    maxwell = mgi.MaxwellCtl( config=maxwell_config)                                       # Turn Maxwell On
    opto_ctl = mgi.OptoCtl( log_file=log_file , 
                       calibration_file=calibration_file, arduino_path=arduino_path)       # Set up Opto
    maxwell.recordingStart( data_file=data_file)                                           # Start recording data             

    print("Starting experiments")
    time.sleep(20)
    for light in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
        print("Intensity: ",light)
        print("Light On")
        opto_ctl.pulses(  on_duration=light_on_seconds*1000, off_duration=100, num_pulses=1, intensity=light)
        print("Light Off")
        time.sleep(30)
    time.sleep(20)
    
    # Shut Down Experiment
    print("Experiment finished. shutting down devices")
    opto_ctl.close()          # Stop Opto
    maxwell.recordingStop()  # stop recording
    print("done")
    
    

## <font color="blue">optoClosedLoop 

In [ ]:
def optoClosedLoop( experiment_length_seconds, count_window_seconds, spike_threshold, light_on_seconds, light_intensity,
                    calibration_file, data_file, log_file  ):
    
    ### Set up Maxwell and Opto
    maxwell = mgi.MaxwellCtl( config=maxwell_config)                                       # Turn Maxwell On
    stream  = mgi.Streamer( maxwell, filtered=False )                                      # Start Livestream
    opto_ctl = mgi.OptoCtl( log_file=log_file , 
                       calibration_file=calibration_file, arduino_path=arduino_path)       # Set up Opto
    maxwell.recordingStart( data_file=data_file)                                           # Start recording data                
    
    time.sleep(30)
    print("ADD BICUCULINE")
    time.sleep(30)
    
    # Define Closed Loop Experiment
    print("Starting closed loop")
    last_opto = 0
    skip_count_check = True
    spike_counts=[]
    experiment_end = time.time() + experiment_length_seconds             # End the experiment in 20 seconds
    while time.time() < experiment_end:                                  # while the experiment hasn't run for 20 seconds
        count_time = time.time() + count_window_seconds                  # Count spikes over 1 second window
        spike_count = 0                                                  # continually add new spikes to spike_count
        while time.time() < count_time:                                  # while still in 1 second window
            frameID, voltages, spikes = stream.getData()                 # get spikes data
            for spike in spikes:                                         # if there's a spike add it to the spike_count
                spike_count += 1
        print("Spike Count: ", spike_count )    
        spike_counts.append(spike_count) 
    
        if spike_count > spike_threshold and not skip_count_check :           # If tehre are more than 10 spikes in a second
            print("Starting Opto Pulses")                                              # Do some opto pulses
            opto_ctl.pulses(  on_duration=light_on_seconds*1000, off_duration=100, num_pulses=1, intensity=light_intensity )
            print("finished pulses")
        else:
            skip_count_check = False
        
    print("experiment finished")    
    time.sleep(60)                                                      # post light recording
    
    # Shut Down Experiment
    print("shutting down devices")
    del stream                # Stop Stream
    opto_ctl.close()          # Stop Opto
    maxwell.recordingStop()  # stop recording
    print("done")
    return spike_counts

In [ ]:
            #skip_count_check =True
            #print("emptying artifact spikes")
            #frameID, voltages, spikes = stream.getData() 
            #last_opto = time.time()
            #empty_time = time.time()+10                                  # wait 10 second to empty out previous data from stream
            #while time.time() < empty_time:
            #    stream.getData()                                         # Empty out stream data, because pulses created many fake spikes
            #print("finished emptying")

# <font color="purple"> Experiment: Opto Intensity Sweep <font color="green"> - Green

## <font color="purple">Run Experiment

In [ ]:
optoIntensitySweep( light_on_seconds=10,
                    log_file= f"./files/opto/logs/{experiment_name}_0",
                    calibration_file= "./files/opto/calibrations/20230308T140514-ucsf-opto-system-b_real-530nm-1a-v1.csv",
                    data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_0.h5" )

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment_name - text about what you did


# <font color="purple"> Experiment: Opto <font color="green">Green</font> Light

## <font color="purple">Run Experiment

In [ ]:
optoRecording( light_on_seconds=10, light_off_seconds=30, num_pulses=3, light_intensity=int("?????????"),
                calibration_file= "./files/opto/calibrations/20230308T140514-ucsf-opto-system-b_real-530nm-1a-v1.csv",
                data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_1.h5",
                log_file = f"./files/opto/logs/{experiment_name}_1")

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment_name - {number}hz {color} light pulse, with {numbers} second pause in between
    * calibration file - ?
    * recording file - ?

# <font color="purple"> Experiment: Opto <font color="red">Red</font> Light

## <font color="purple">Run Experiment

In [ ]:
optoRecording( light_on_seconds=10, light_off_seconds=30, num_pulses=3, light_intensity=int("???????"),
                calibration_file= "./files/opto/calibrations/20230203T152939-ucsf-opto-system-a_real-625nm-1a-v1.csv",
                data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_2.h5",
                log_file = f"./files/opto/logs/{experiment_name}_2")

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment_name - {number}hz {color} light pulse, with {numbers} second pause in between
    * calibration file - ?
    * recording file - ?

# <font color="purple"> Experiment: Opto <font color="blue">Blue</font> Light

## <font color="purple">Run Experiment

In [ ]:
optoRecording( light_on_seconds=10, light_off_seconds=30, num_pulses=3, light_intensity=int("????????"),
                calibration_file= "./files/opto/calibrations/20230203T153451-ucsf-opto-system-b_real-470nm-1a-v1.csv",
                data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_3.h5",
                log_file = f"./files/opto/logs/{experiment_name}_3")

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment_name - {number}hz {color} light pulse, with {numbers} second pause in between
    * calibration file - ?
    * recording file - ?

# <font color="purple"> Experiment: Count Spikes

## <font color="purple">Run Experiment

In [12]:
spike_counts = countSpikes(30)

starting devices
MaxOne initialized
maxwell streamer ready
Starting experiment
Spike Count:  59
Spike Count:  69
Spike Count:  69
Experiment over. Shutting down devices
done


## <font color="purple">Analyze Results

Plot histogram of spikes

In [ ]:
print("Mean Spikes: ", np.mean(spike_counts) )
plt.hist(spike_counts, bins='auto')  # arguments are passed to np.histogram
plt.show()

Save Data

In [ ]:
np.savetxt( experiment_name+"_calibrate_spike_counts.csv", spike_counts, delimiter="," )
#checkit = np.loadtxt("some_experiment_spike_counts.csv")   # How to load data

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment 1 - text about what you did
    * experiment_length_seconds=?, count_window_seconds=?, csv_file=?

# <font color="purple"> Experiment: Opto Closed Loop - No Bicuculine <font color="green">

## <font color="purple">Run Experiment

<font color="red"><b>IMportant</b> how long do you want to record after light stim

Run experiment

In [ ]:
spike_counts = optoClosedLoop( experiment_length_seconds= 60,
                count_window_seconds=10,
                spike_threshold= int("???"),
                light_on_seconds= 10,
                light_intensity= int("???"),
                calibration_file= "./files/opto/calibrations/20230308T140514-ucsf-opto-system-b_real-530nm-1a-v1.csv",
                data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_4.h5",
                log_file=  f"./files/opto/logs/{experiment_name}_4" )

In [ ]:
spike_counts = optoClosedLoop( experiment_length_seconds= 40,
                count_window_seconds=10,
                spike_threshold= 40, #int("???"),
                light_on_seconds= 10,
                light_intensity= .9, #int("???"),
                calibration_file= "./files/opto/calibrations/20230308T140514-ucsf-opto-system-b_real-530nm-1a-v1.csv",
                data_file= f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_9.h5",
                log_file=  f"./files/opto/logs/{experiment_name}_9" )

## <font color="purple">Document experiment

Save spike counts

In [ ]:
np.savetxt( experiment_name+"_no_bicuculine_spike_counts.csv", spike_counts, delimiter="," )

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment 1 - text about what you did
    * experiment_length_seconds=?, count_window_seconds=?, csv_file=?

# <font color="purple"> Experiment: Closed Loop - <font color="red">Bicuculine

## <font color="purple">Run Experiment

Start Recording

In [ ]:
# Set variables
experiment_length_seconds = 120
count_window_seconds = 10
spike_threshold  = 40 #270 
light_on_seconds = 10
light_intensity  = .5
calibration_file = "./files/opto/calibrations/20230308T140514-ucsf-opto-system-b_real-530nm-1a-v1.csv"
data_file = f" /home/mxwbio/work/seizure_closed_loop/files/maxwell/recordings/{experiment_name}_5.h5"
log_file  = f"./files/opto/logs/{experiment_name}_5" 


### Set up Maxwell and Opto
maxwell = mgi.MaxwellCtl( config=maxwell_config)                                       # Turn Maxwell On
stream  = mgi.Streamer( maxwell, filtered=False )                                      # Start Livestream
opto_ctl = mgi.OptoCtl( log_file=log_file , 
                    calibration_file=calibration_file, arduino_path=arduino_path)       # Set up Opto
maxwell.recordingStart( data_file=data_file)                                           # Start recording data                


Start recording

right after bicuculinw is added, run closed loop

In [ ]:
experiment_length_seconds = 40

In [ ]:
# Define Closed Loop Experiment
print("Starting closed loop")
last_opto = 0
first_time = True
spike_counts=[]
experiment_end = time.time() + experiment_length_seconds             # End the experiment in 20 seconds
while time.time() < experiment_end:                                  # while the experiment hasn't run for 20 seconds
    count_time = time.time() + count_window_seconds                  # Count spikes over 1 second window
    spike_count = 0                                                  # continually add new spikes to spike_count
    while time.time() < count_time:                                  # while still in 1 second window
        frameID, voltages, spikes = stream.getData()                 # get spikes data
        for spike in spikes:                                         # if there's a spike add it to the spike_count
            spike_count += 1
    print("Spike Count: ", spike_count )    
    spike_counts.append(spike_count) 
    
    if spike_count > spike_threshold and time.time()>last_opto+light_on_seconds+5  and not first_time:           # If tehre are more than 10 spikes in a second
        print("Starting Opto Pulses")                                              # Do some opto pulses
        opto_ctl.pulses(  on_duration=light_on_seconds*1000, off_duration=100, num_pulses=1, intensity=light_intensity )
        print("finished pulses")
            #print("emptying artifact spikes")
        #stream.getData()  
        last_opto = time.time()
        stream.getData()  
            #empty_time = time.time()+10                                  # wait 10 second to empty out previous data from stream
            #while time.time() < empty_time:
            #    stream.getData()                                         # Empty out stream data, because pulses created many fake spikes
            #print("finished emptying")
    first_time = False
        
print("experiment finished")    
#time.sleep(120)

stop experiment

In [ ]:
# Shut Down Experiment
print("shutting down devices")
del stream                # Stop Stream
opto_ctl.close()          # Stop Opto
maxwell.recordingStop()  # stop recording
print("done")

## <font color="purple">Document experiment

<font color="red">Use the markdown below to document your experiment

**Experiments**

* Experiment_name -

# <font color="gray"> Additional Experiment Notes

Please put any additional experiment notes here